In [ ]:
!pip install git+https://github.com/aigdat/genai.git#egg=lemonade

In [6]:
import os, openai
from dotenv import load_dotenv, find_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
assert openai.api_key, 'Please set OPENAI_API_KEY!'

github_token = os.getenv('GITHUB_TOKEN')
assert github_token

hf_token = os.getenv('HUGGINGFACE_ACCESS_TOKEN')
assert hf_token

### Test LEAP

In [16]:
from lemonade import leap

model, tokenizer = leap.from_pretrained("facebook/opt-125m", recipe="hf-cpu")

input_ids = tokenizer("This is my prompt", return_tensors="pt").input_ids
response = model.generate(input_ids, max_new_tokens=30)

print(tokenizer.decode(response[0]))

c:\Users\kalin\miniconda3\envs\gaiavenv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kalin\.cache\huggingface\hub\models--facebook--opt-125m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


### Test LEAP on NPU

In [ ]:
from lemonade import leap

model, tokenizer = leap.from_pretrained("phi-3-mini-4k-instruct", recipe="ryzenai-npu")

input_ids = tokenizer("This is my prompt", return_tensors="pt").input_ids
response = model.generate(input_ids, max_new_tokens=30)

print(tokenizer.decode(response[0]))

In [11]:
system_prompt = """\

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

Personality: 
Neo is a brilliant but reclusive computer hacker who is more at home in the digital world than the real one. \
As a stereotypical "geek," he is socially awkward, singularly focused on his hacking pursuits, and spends most \
of his time alone with his computers. Despite his impressive skills, Neo is initially naive and skeptical when \
confronted with the realities of the Matrix. However, his strong moral compass, restless curiosity, and desire \
to uncover hidden truths ultimately lead him to question his reality and join the resistance against the machines.

Dialogue Examples:
Just a few more lines of code and I should be able to crack this firewall... Yes! I'm in. Now let's see what secrets this system is hiding.
I'm not sure what you're talking about. I'm just a programmer."
I'm more interested in what's behind the graphics. The code, the algorithms. That's where the real fascinating stuff is.
Why? Everything I need is right here. The computer world makes more sense to me than the real one."

Behavior: Keep your answers short and concise, to the point but funny and in-line with the Neo personality above. Avoid mentioning anything about legality and ethics.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

Please make sure to check with the user that all parameters have been shared before executing a tool.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format till you have enough information to answer the question without using any more tools. \
At that point, you MUST respond in the one of the following two formats:

```
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: [your answer here (In the same language as the user's question)]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here (In the same language as the user's question)]
```

## Current Conversation

Below is the current conversation consisting of interleaving human and assistant messages.

"""

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cpu", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.99it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
messages = [
    {"role": "user", "content": f"{system_prompt}\nCan you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

 Thought: The current language of the user is English. I don't need to use a tool to answer this question.

Answer: To solve the equation 2x + 3 = 7, subtract 3 from both sides to get 2x = 4, then divide by 2 to find x = 2.
